# Imports

In [39]:
import requests
import json
import pandas as pd
import time
from time import sleep
import random
from random import randint
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', None)


# Extract info from website :

In [40]:
%%time

pages = []
for i in range(1,5000):

    response = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        'currency_code':'EUR',
        'grape_filter':'varietal',
        'min_rating':'3.6',
        'order_by':'ratings_average',
        'order':'desc',
        'wine_type_ids[]':'1',
        'page':i,
        'language':'en'
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)

    #store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,2000)
    sleep(wait_time/1000)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [41]:
len(pages)

2409

In [42]:
# Nr. of wines
response.json()['explore_vintage']['records_matched']

25735

In [43]:
response.json()['explore_vintage']['matches'][0].keys()

dict_keys(['vintage', 'price', 'prices'])

## Vintage

In [44]:
all_wines = []
vintage = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        vintage.append(all_wines[i][j]['vintage'])

In [45]:
vintage_df = pd.DataFrame(vintage)
vintage_df.head(3)

,id,seo_name,name,statistics,image,wine,year,grapes,has_valid_ratings,top_list_rankings
0,8519046,franco-biondi-santi-brunello-di-montalcino-ris...,Biondi-Santi Brunello di Montalcino Riserva 1955,"{'status': 'Normal', 'ratings_count': 33, 'rat...",{'location': '//images.vivino.com/thumbs/C62n7...,"{'id': 82698, 'name': 'Brunello di Montalcino ...",1955,None,True,NaN
1,2890763,quintarelli-giuseppe-amarone-della-valpolicell...,Quintarelli Giuseppe Amarone della Valpolicell...,"{'status': 'Normal', 'ratings_count': 311, 'ra...",{'location': '//images.vivino.com/thumbs/RqFtS...,"{'id': 84065, 'name': 'Amarone della Valpolice...",2013,None,True,NaN
2,14245513,vega-sicilia-unico-reserva-especial-edicion-2015,Vega Sicilia Unico Reserva Especial Edición 2015,"{'status': 'Normal', 'ratings_count': 644, 'ra...",{'location': '//images.vivino.com/thumbs/k_Uet...,"{'id': 77136, 'name': 'Unico Reserva Especial ...",2015,None,True,NaN


In [46]:
vintage_df['id'].nunique()

2026

In [47]:
#vintage_df['top_list_rankings'].value_counts(dropna = False)

#### Remove unnecessary columns

 - 'statistics' and 'wine' info will be extracted separetely, therefore I will drop these columns as well;
 - 'image' and 'top_list_rankings' are not required for the model
 - 'grapes' only has None values. This info is captured under wines section
 - 'has_valid_ratings' is always 'True'

In [48]:
vintage_df = vintage_df.drop(['statistics','image','wine','grapes','has_valid_ratings','top_list_rankings'], axis = 1)
vintage_df

,id,seo_name,name,year
0,8519046,franco-biondi-santi-brunello-di-montalcino-ris...,Biondi-Santi Brunello di Montalcino Riserva 1955,1955
1,2890763,quintarelli-giuseppe-amarone-della-valpolicell...,Quintarelli Giuseppe Amarone della Valpolicell...,2013
2,14245513,vega-sicilia-unico-reserva-especial-edicion-2015,Vega Sicilia Unico Reserva Especial Edición 2015,2015
3,159478830,vega-sicilia-unico-reserva-especial-edicion-2019,Vega Sicilia Unico Reserva Especial Edición 2019,2019
4,2712391,domaine-armand-rousseau-chambertin-clos-de-bez...,Domaine Armand Rousseau Chambertin Clos de Bez...,1995
...,...,...,...,...
60220,166769843,coppo-nizza-bric-del-marchese-2017,Coppo Nizza 'Bric del Marchese' 2017,2017
60221,145631458,ceretto-barolo-bricco-rocche-2016,Ceretto Barolo Bricco Rocche 2016,2016
60222,156258898,anakota-wines-helena-dakota-vineyard-cabernet-...,Anakota Helena Dakota Vineyard Cabernet Sauvig...,2018
60223,156464165,francis-ford-coppola-winery-archimedes-2018,Francis Ford Coppola Winery Archimedes 2018,2018


## Statistics

In [49]:
all_wines = []
statistics = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        statistics.append(all_wines[i][j]['vintage']['statistics'])

In [50]:
statistics_df = pd.DataFrame(statistics)
statistics_df.head(3)

,status,ratings_count,ratings_average,labels_count,wine_ratings_count,wine_ratings_average,wine_status
0,Normal,33,4.9,122,3302,4.6,Normal
1,Normal,311,4.8,2310,8465,4.7,Normal
2,Normal,644,4.8,3614,12819,4.7,Normal


In [51]:
len(statistics_df['status'])

60225

#### Remove unnecessary columns

In [52]:
# for col in statistics_df.columns:
#     #print(customer_df[col].unique)
#     print('Name: {}'.format(statistics_df[col].name))    # column name
#     print('Type: {}'.format(statistics_df[col].dtype))   # column type
#     print('Unique values: {}'.format(len(statistics_df[col].unique())))   # column unique values
#     print('% Unique values: {}'.format(round(len(statistics_df[col].unique())/len(statistics_df)*100,1)))  # percentage
#     print('\n----------')
    

In [53]:
statistics_df = statistics_df.drop(['status','labels_count','wine_ratings_count','wine_ratings_average','wine_status'], axis = 1)
statistics_df

,ratings_count,ratings_average
0,33,4.9
1,311,4.8
2,644,4.8
3,256,4.8
4,30,4.9
...,...,...
60220,33,4.5
60221,33,4.5
60222,33,4.5
60223,33,4.5


## Wine info

In [54]:
all_wines = []
wine = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        wine.append(all_wines[i][j]['vintage']['wine'])

In [55]:
wine_df = pd.DataFrame(wine)

wine_df.columns = 'wine_' + wine_df.columns
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Amarone della Valpolicel...","{'id': 11601, 'name': 'Quintarelli Giuseppe', ...","{'structure': {'acidity': 2.3577335, 'fizzines...","{'status': 'Normal', 'ratings_count': 8465, 'r...","{'id': 3, 'seo_name': 'italian-amarone', 'regi...",True
2,77136,Unico Reserva Especial Edición,unico-reserva-especial-edicion,1,2,False,"{'id': 405, 'name': 'Ribera del Duero', 'name_...","{'id': 11050, 'name': 'Vega Sicilia', 'seo_nam...","{'structure': {'acidity': 3.6732104, 'fizzines...","{'status': 'Normal', 'ratings_count': 12819, '...","{'id': 180, 'seo_name': 'spanish-ribera-del-du...",True


### wine_region

In [56]:
#Flat columns
wine_region = pd.json_normalize(wine_df['wine_region'])
wine_region.columns = 'wine_region_' + wine_region.columns
wine_region.shape

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_region], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_region'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(60225, 33)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Amarone della Valpolicel...","{'id': 11601, 'name': 'Quintarelli Giuseppe', ...","{'structure': {'acidity': 2.3577335, 'fizzines...","{'status': 'Normal', 'ratings_count': 8465, 'r...","{'id': 3, 'seo_name': 'italian-amarone', 'regi...",True,3237,Amarone della Valpolicella Classico,,amarone-della-valpolicella-classico,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/Z4sEIs...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,NaN
2,77136,Unico Reserva Especial Edición,unico-reserva-especial-edicion,1,2,False,"{'id': 405, 'name': 'Ribera del Duero', 'name_...","{'id': 11050, 'name': 'Vega Sicilia', 'seo_nam...","{'structure': {'acidity': 3.6732104, 'fizzines...","{'status': 'Normal', 'ratings_count': 12819, '...","{'id': 180, 'seo_name': 'spanish-ribera-del-du...",True,405,Ribera del Duero,,ribera-del-duero,es,Spanje,España,spain,EUR,Euros,€,None,152,2153435,138476,17963,"[{'id': 19, 'name': 'Tempranillo', 'seo_name':...",//images.vivino.com/regions/backgrounds/_nMNiD...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,NaN


### wine_winery

In [57]:
#Flat columns
wine_winery = pd.json_normalize(wine_df['wine_winery'])
wine_winery.columns = 'wine_winery_' + wine_winery.columns
# print(wine_winery.shape)
# wine_winery

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_winery], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(60225, 42)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN,11500,Biondi-Santi,franco-biondi-santi,0,NaN,NaN,NaN,NaN,NaN
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Amarone della Valpolicel...","{'id': 11601, 'name': 'Quintarelli Giuseppe', ...","{'structure': {'acidity': 2.3577335, 'fizzines...","{'status': 'Normal', 'ratings_count': 8465, 'r...","{'id': 3, 'seo_name': 'italian-amarone', 'regi...",True,3237,Amarone della Valpolicella Classico,,amarone-della-valpolicella-classico,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/Z4sEIs...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,NaN,11601,Quintarelli Giuseppe,quintarelli-giuseppe,0,NaN,NaN,NaN,NaN,NaN
2,77136,Unico Reserva Especial Edición,unico-reserva-especial-edicion,1,2,False,"{'id': 405, 'name': 'Ribera del Duero', 'name_...","{'id': 11050, 'name': 'Vega Sicilia', 'seo_nam...","{'structure': {'acidity': 3.6732104, 'fizzines...","{'status': 'Normal', 'ratings_count': 12819, '...","{'id': 180, 'seo_name': 'spanish-ribera-del-du...",True,405,Ribera del Duero,,ribera-del-duero,es,Spanje,España,spain,EUR,Euros,€,None,152,2153435,138476,17963,"[{'id': 19, 'name': 'Tempranillo', 'seo_name':...",//images.vivino.com/regions/backgrounds/_nMNiD...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,NaN,11050,Vega Sicilia,vega-sicilia,0,NaN,NaN,NaN,NaN,NaN


### wine_taste

In [58]:
#Flat columns
wine_taste = pd.json_normalize(wine_df['wine_taste'])
wine_taste.columns = 'wine_taste_' + wine_taste.columns
#print(wine_taste.shape)
#wine_taste

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_taste], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(60225, 51)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_flavor,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN,11500,Biondi-Santi,franco-biondi-santi,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'earth', 'stats': {'count': 129, 's...",3.811572,NaN,3.170349,1.678928,3.586557,158.0,163.0,NaN
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Amarone della Valpolicel...","{'id': 11601, 'name': 'Quintarelli Giuseppe', ...","{'structure': {'acidity': 2.3577335, 'fizzines...","{'status': 'Normal', 'ratings_count': 8465, 'r...","{'id': 3, 'seo_name': 'italian-amarone', 'regi...",True,3237,Amarone della Valpolicella Classico,,amarone-della-valpolicella-classico,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/Z4sEIs...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,NaN,11601,Quintarelli Giuseppe,quintarelli-giuseppe,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'oak', 'stats': {'count': 252, 'sco...",2.357734,NaN,4.611889,3.318892,2.318914,327.0,422.0,NaN
2,77136,Unico Reserva Especial Edición,unico-reserva-especial-edicion,1,2,False,"{'id': 405, 'name': 'Ribera del Duero', 'name_...","{'id': 11050, 'name': 'Vega Sicilia', 'seo_nam...","{'structure': {'acidity': 3.6732104, 'fizzines...","{'status': 'Normal', 'ratings_count': 12819, '...","{'id': 180, 'seo_name': 'spanish-ribera-del-du...",True,405,Ribera del Duero,,ribera-del-duero,es,Spanje,España,spain,EUR,Euros,€,None,152,2153435,138476,17963,"[{'id': 19, 'name': 'Tempranillo', 'seo_name':...",//images.vivino.com/regions/backgrounds/_nMNiD...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,//thumbs.vivino.com/region_backgrounds/_nMNiDR...,NaN,11050,Vega Sicilia,vega-sicilia,0,NaN,NaN,NaN,NaN,NaN,"[{'group': 'oak', 'stats': {'count': 252, 'sco...",3.673210,NaN,3.852942,1.838485,3.483987,496.0,322.0,NaN


#### wine_taste_flavor

In [59]:
#Flat columns
wine_flavor = pd.json_normalize(wine_df['wine_taste_flavor'])


# #Flat the first 4 columns from flavor table. This are most voted tastes by users
wine_flavor_1 = pd.json_normalize(wine_flavor[0])
wine_flavor_1.columns = 'wine_taste_flavor_1' + wine_flavor_1.columns
wine_flavor_2 = pd.json_normalize(wine_flavor[1])
wine_flavor_2.columns = 'wine_taste_flavor_2' + wine_flavor_2.columns
wine_flavor_3 = pd.json_normalize(wine_flavor[2])
wine_flavor_3.columns = 'wine_taste_flavor_3' + wine_flavor_3.columns
wine_flavor_4 = pd.json_normalize(wine_flavor[3])
wine_flavor_4.columns = 'wine_taste_flavor_4' + wine_flavor_4.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_flavor_1, wine_flavor_2, wine_flavor_3, wine_flavor_4], axis=1)
wine_df = wine_df.drop('wine_taste_flavor', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1primary_keywords,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN,11500,Biondi-Santi,franco-biondi-santi,0,NaN,NaN,NaN,NaN,NaN,3.811572,NaN,3.170349,1.678928,3.586557,158.0,163.0,NaN,earth,"[{'id': 242, 'name': 'leather', 'count': 57}, ...","[{'id': 422, 'name': 'tobacco', 'count': 45}, ...",129.0,19105.0,oak,"[{'id': 422, 'name': 'tobacco', 'count': 45}, ...","[{'id': 242, 'name': 'leather', 'count': 57}, ...",114.0,12427.0,red_fruit,"[{'id': 93, 'name': 'cherry', 'count': 48}, {'...","[{'id': 229, 'name': 'jam', 'count': 7}, {'id'...",96.0,11998.0,non_oak,"[{'id': 417, 'name': 'toast', 'count': 2}, {'i...","[{'id': 422, 'name': 'tobacco', 'count': 45}, ...",86.0,2446.0
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Amarone della Valpolicel...","{'id': 11601, 'name': 'Quintarelli Giuseppe', ...","{'structure': {'acidity': 2.3577335, 'fizzines...","{'status': 'Normal', 'ratings_count': 8465, 'r...","{'id': 3, 'seo_name': 'italian-amarone', 'regi...",True,3237,Amarone della Valpolicella Classico,,amarone-della-valpolicella-classico,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/Z4sEIs...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,//thumbs.vivino.com/region_backgrounds/Z4sEIsQ...,NaN,11601,Quinta

#### wine_taste_flavor_keywords

In [60]:
wine_taste_keyword1 = pd.json_normalize(wine_df['wine_taste_flavor_1primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword1_1 = pd.json_normalize(wine_taste_keyword1[0])
wine_taste_keyword1_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword1_1.columns
wine_taste_keyword1_2 = pd.json_normalize(wine_taste_keyword1[1])
wine_taste_keyword1_2.columns = 'wine_taste_flavor_2primary_keywords' + wine_taste_keyword1_2.columns
wine_taste_keyword1_3 = pd.json_normalize(wine_taste_keyword1[2])
wine_taste_keyword1_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword1_3.columns
wine_taste_keyword1_4 = pd.json_normalize(wine_taste_keyword1[3])
wine_taste_keyword1_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword1_4.columns
wine_taste_keyword1_5 = pd.json_normalize(wine_taste_keyword1[4])
wine_taste_keyword1_5.columns = 'wine_taste_flavor_5primary_keywords' + wine_taste_keyword1_5.columns



#combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword1_1, wine_taste_keyword1_2, wine_taste_keyword1_3, wine_taste_keyword1_4, wine_taste_keyword1_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_1primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2primary_keywords,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN,11500,Biondi-Santi,franco-biondi-santi,0,NaN,NaN,NaN,NaN,NaN,3.811572,NaN,3.170349,1.678928,3.586557,158.0,163.0,NaN,earth,"[{'id': 422, 'name': 'tobacco', 'count': 45}, ...",129.0,19105.0,oak,"[{'id': 422, 'name': 'tobacco', 'count': 45}, ...","[{'id': 242, 'name': 'leather', 'count': 57}, ...",114.0,12427.0,red_fruit,"[{'id': 93, 'name': 'cherry', 'count': 48}, {'...","[{'id': 229, 'name': 'jam', 'count': 7}, {'id'...",96.0,11998.0,non_oak,"[{'id': 417, 'name': 'toast', 'count': 2}, {'i...","[{'id': 422, 'name': 'tobacco', 'count': 45}, ...",86.0,2446.0,242.0,leather,57.0,284.0,mushroom,40.0,156.0,earthy,26.0,430.0,truffle,14.0,384.0,smoke,14.0
1,84065,Amarone della Valpolicella Classico,amarone-della-valpolicella-classico,1,0,False,"{'id': 3237, 'name': 'Ama

In [61]:
wine_taste_keyword2 = pd.json_normalize(wine_df['wine_taste_flavor_2primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword2_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword2_1.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_1.columns
wine_taste_keyword2_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword2_2.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_2.columns
wine_taste_keyword2_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword2_3.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_3.columns
wine_taste_keyword2_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword2_4.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_4.columns
wine_taste_keyword2_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword2_5.columns = 'wine_taste_flavor_1primary_keywords' + wine_taste_keyword2_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword2_1, wine_taste_keyword2_2, wine_taste_keyword2_3, wine_taste_keyword2_4, wine_taste_keyword2_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_2primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3primary_keywords,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-brunello', 're...",True,1796,Brunello di Montalcino,,brunello-di-montalcino,it,Italië,Italia,italy,EUR,Euros,€,None,560,4071126,372183,42165,"[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...",//images.vivino.com/regions/backgrounds/c25PzK...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,//thumbs.vivino.com/region_backgrounds/c25PzK2...,NaN,11500,Biondi-Santi,franco-biondi-santi,0,NaN,NaN,NaN,NaN,NaN,3.811572,NaN,3.170349,1.678928,3.586557,158.0,163.0,NaN,earth,"[{'id': 422, 'name': 'tobacco', 'count': 45}, ...",129.0,19105.0,oak,"[{'id': 242, 'name

In [62]:
wine_taste_keyword3 = pd.json_normalize(wine_df['wine_taste_flavor_3primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword3_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword3_1.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_1.columns
wine_taste_keyword3_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword3_2.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_2.columns
wine_taste_keyword3_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword3_3.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_3.columns
wine_taste_keyword3_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword3_4.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_4.columns
wine_taste_keyword3_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword3_5.columns = 'wine_taste_flavor_3primary_keywords' + wine_taste_keyword3_5.columns



# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword3_1, wine_taste_keyword3_2, wine_taste_keyword3_3, wine_taste_keyword3_4, wine_taste_keyword3_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_3primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4primary_keywords,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount
0,82698,Brunello di Montalcino Riserva,brunello-di-montalcino-riserva,1,0,False,"{'id': 1796, 'name': 'Brunello di Montalcino',...","{'id': 11500, 'name': 'Biondi-Santi', 'seo_nam...","{'structure': {'acidity': 3.8115718, 'fizzines...","{'status': 'Normal', 'ratings_count': 3302, 'r...","{'id': 22, 'seo_name': 'italian-

In [63]:
wine_taste_keyword4 = pd.json_normalize(wine_df['wine_taste_flavor_4primary_keywords'])

# #Flat the first 10 columns from primary keywords table
wine_taste_keyword4_1 = pd.json_normalize(wine_taste_keyword2[0])
wine_taste_keyword4_1.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_1.columns
wine_taste_keyword4_2 = pd.json_normalize(wine_taste_keyword2[1])
wine_taste_keyword4_2.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_2.columns
wine_taste_keyword4_3 = pd.json_normalize(wine_taste_keyword2[2])
wine_taste_keyword4_3.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_3.columns
wine_taste_keyword4_4 = pd.json_normalize(wine_taste_keyword2[3])
wine_taste_keyword4_4.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_4.columns
wine_taste_keyword4_5 = pd.json_normalize(wine_taste_keyword2[4])
wine_taste_keyword4_5.columns = 'wine_taste_flavor_4primary_keywords' + wine_taste_keyword4_5.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_taste_keyword4_1, wine_taste_keyword4_2, wine_taste_keyword4_3, wine_taste_keyword4_4, wine_taste_keyword4_5], axis=1)
wine_df = wine_df.drop('wine_taste_flavor_4primary_keywords', axis=1)
wine_df.shape
wine_df.head(3)

,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_statistics

In [64]:
#Flat columns
wine_statistics = pd.json_normalize(wine_df['wine_statistics'])
wine_statistics.columns = 'wine_statistics_' + wine_statistics.columns
# print(wine_statistics.shape)
# wine_statistics

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_statistics], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(60225, 131)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

### wine_style

In [65]:
#Flat columns
wine_style = pd.json_normalize(wine_df['wine_style'])
wine_style.columns = 'wine_style_' + wine_style.columns
# print(wine_style.shape)
# wine_style

# combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df,wine_style], axis = 1)
#wine_df = wine_df.drop(wine_df['wine_winery'], axis = 1)
print(wine_df.shape)
wine_df.head(3)

(60225, 190)


,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,

#### wine_style_food

In [66]:
#Flat columns
wine_food = pd.json_normalize(wine_df['wine_style_food'])

#Flat each column from food table
wine_food_1 = pd.json_normalize(wine_food[0])
wine_food_1.columns = 'wine_food_' + wine_food_1.columns
wine_food_2 = pd.json_normalize(wine_food[1])
wine_food_2.columns = 'wine_food_' + wine_food_2.columns
wine_food_3 = pd.json_normalize(wine_food[2])
wine_food_3.columns = 'wine_food_' + wine_food_3.columns
wine_food_4 = pd.json_normalize(wine_food[3])
wine_food_4.columns = 'wine_food_' + wine_food_4.columns
wine_food_5 = pd.json_normalize(wine_food[4])
wine_food_5.columns = 'wine_food_' + wine_food_5.columns


# #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_food_1, wine_food_2, wine_food_3, wine_food_4, wine_food_5], axis=1)
wine_df = wine_df.drop('wine_style_food', axis=1)
wine_df.shape

(60225, 214)

#### wine_style_grapes

In [67]:
#Flat columns
wine_grapes = pd.json_normalize(wine_df['wine_style_grapes'])

# #Flat each column from food table
wine_grapes_1 = pd.json_normalize(wine_grapes[0])
wine_grapes_1.columns = 'wine_style_grapes' + wine_grapes_1.columns
wine_grapes_2 = pd.json_normalize(wine_grapes[1])
wine_grapes_2.columns = 'wine_style_grapes_' + wine_grapes_2.columns
wine_grapes_3 = pd.json_normalize(wine_grapes[2])
wine_grapes_3.columns = 'wine_style_grapes_' + wine_grapes_3.columns
wine_grapes_4 = pd.json_normalize(wine_grapes[3])
wine_grapes_4.columns = 'wine_style_grapes_' + wine_grapes_4.columns
wine_grapes_5 = pd.json_normalize(wine_grapes[4])
wine_grapes_5.columns = 'wine_style_grapes_' + wine_grapes_5.columns
wine_grapes_6 = pd.json_normalize(wine_grapes[5])
wine_grapes_6.columns = 'wine_style_grapes_' + wine_grapes_6.columns


# # #combine all dfs. drop original column, keep flattened
wine_df = pd.concat([wine_df, wine_grapes_1, wine_grapes_2, wine_grapes_3, wine_grapes_4, wine_grapes_5, wine_grapes_6], axis=1)
wine_df = wine_df.drop('wine_style_grapes', axis=1)
wine_df.shape

(60225, 243)

## Price

In [68]:
all_wines = []
prices = []
for i in range(len(pages)):
    all_wines.append(pages[i].json()['explore_vintage']['matches'])
    for j in range(len(all_wines[i])):
        prices.append(all_wines[i][j]['price'])

In [69]:
prices_df = pd.DataFrame(prices)
prices_df.head(3)

,id,amount,discounted_from,discount_percent,type,sku,url,visibility,bottle_type_id,currency,bottle_type
0,21578167,1970.05,NaN,None,vc,36c988be-60fe-4536-98bf-3bc7f00f0423,https://hermanwines.com/products/brunello-rise...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
1,30639671,389.00,NaN,None,vc,AMA-1010-R03807-13,https://www.colaris.nl/quintarelli-giuseppe-am...,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."
2,23399625,608.32,NaN,None,vc,7199,http://cultwijnen.nl/,1,1,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'...","{'id': 1, 'name': 'Fles (0.75 l)', 'short_name..."


#### Remove unnecessary columns

Price information will be used to improve recommendatio. Only the price per bottle is relevant

In [70]:
drop_lst = ['id','type','discounted_from','discount_percent','sku','url','visibility','bottle_type_id','bottle_type']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,currency
0,1970.05,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
1,389.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
2,608.32,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
3,593.07,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
4,4228.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
...,...,...
60220,66.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
60221,200.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
60222,129.00,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."
60223,79.95,"{'code': 'EUR', 'name': 'Euros', 'prefix': '€'..."


In [71]:
#Flat columns
currency = pd.json_normalize(prices_df['currency'])

# # #combine all dfs. drop original column, keep flattened
prices_df = pd.concat([prices_df, currency], axis=1)
prices_df = prices_df.drop('currency', axis=1)
prices_df.shape

# keep only currency code
drop_lst = ['name','prefix','suffix']
prices_df = prices_df.drop(drop_lst, axis = 1)
prices_df

,amount,code
0,1970.05,EUR
1,389.00,EUR
2,608.32,EUR
3,593.07,EUR
4,4228.95,EUR
...,...,...
60220,66.00,EUR
60221,200.00,EUR
60222,129.00,EUR
60223,79.95,EUR


## Make full df from all tables above

In [72]:
all_wines_df = pd.concat([vintage_df,prices_df,statistics_df,wine_df], axis = 1)
print(all_wines_df.shape)
all_wines_df.head(3)

(60225, 251)


,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_region,wine_winery,wine_taste,wine_statistics,wine_style,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor

In [73]:
len(all_wines_df)

60225

In [74]:
columns_to_drop = ['wine_region','wine_winery','wine_taste','wine_statistics','wine_style']

In [75]:
all_wines_df = all_wines_df.drop(columns_to_drop, axis = 1)
all_wines_df

,id,seo_name,name,year,amount,code,ratings_count,ratings_average,wine_id,wine_name,wine_seo_name,wine_type_id,wine_vintage_type,wine_is_natural,wine_has_valid_ratings,wine_region_id,wine_region_name,wine_region_name_en,wine_region_seo_name,wine_region_country.code,wine_region_country.name,wine_region_country.native_name,wine_region_country.seo_name,wine_region_country.currency.code,wine_region_country.currency.name,wine_region_country.currency.prefix,wine_region_country.currency.suffix,wine_region_country.regions_count,wine_region_country.users_count,wine_region_country.wines_count,wine_region_country.wineries_count,wine_region_country.most_used_grapes,wine_region_background_image.location,wine_region_background_image.variations.large,wine_region_background_image.variations.medium,wine_region_background_image,wine_winery_id,wine_winery_name,wine_winery_seo_name,wine_winery_status,wine_winery_background_image,wine_winery_background_image.location,wine_winery_background_image.variations.large,wine_winery_background_image.variations.medium,wine_winery_background_image.variations.small,wine_taste_structure.acidity,wine_taste_structure.fizziness,wine_taste_structure.intensity,wine_taste_structure.sweetness,wine_taste_structure.tannin,wine_taste_structure.user_structure_count,wine_taste_structure.calculated_structure_count,wine_taste_structure,wine_taste_flavor_1group,wine_taste_flavor_1secondary_keywords,wine_taste_flavor_1stats.count,wine_taste_flavor_1stats.score,wine_taste_flavor_2group,wine_taste_flavor_2secondary_keywords,wine_taste_flavor_2stats.count,wine_taste_flavor_2stats.score,wine_taste_flavor_3group,wine_taste_flavor_3secondary_keywords,wine_taste_flavor_3stats.count,wine_taste_flavor_3stats.score,wine_taste_flavor_4group,wine_taste_flavor_4secondary_keywords,wine_taste_flavor_4stats.count,wine_taste_flavor_4stats.score,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_2primary_keywordsid,wine_taste_flavor_2primary_keywordsname,wine_taste_flavor_2primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_5primary_keywordsid,wine_taste_flavor_5primary_keywordsname,wine_taste_flavor_5primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_1primary_keywordsid,wine_taste_flavor_1primary_keywordsname,wine_taste_flavor_1primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_3primary_keywordsid,wine_taste_flavor_3primary_keywordsname,wine_taste_flavor_3primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscount,wine_taste_flavor_4primary_keywordsid,wine_taste_flavor_4primary_keywordsname,wine_taste_flavor_4primary_keywordscoun

In [76]:
all_wines_df.to_csv('all_wines_df_type1_v3.csv')